In [ ]:
## Cell 1
## Extract positive pairs for training
import numpy as np
import PIL
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import random

save_path = ''
data_path = ''
disp_path = ''
list_path = ''
folders = ['20170221_1357', '20170222_0715', '20170222_1207', '20170222_1638', '20170223_0920', '20170223_1217', '20170223_1445', '20170224_1022']
size = 32
records = []
for folder in folders:
    f = open(Path(list_path) / (folder + '.txt'), 'r')
    lines = f.readlines()
    f.close()
    for i, line in enumerate(lines):
        splited = line.split()
        collection = splited[0]
        key = splited[1]
        rgb_exp = float(splited[2])
        nir_exp = float(splited[3])
        record = [collection, key, rgb_exp, nir_exp]
        records.append(record)

cnt = 0
for record in records:
    path_load_c_l = data_path + '/' + record[0] + '/RGBResize/' + record[1] + '_RGBResize.png'
    path_load_c_r = data_path + '/' + record[0] + '/NIRResize/' + record[1] + '_NIRResize.png'
    path_load_disp_l = disp_path + '/left_pngs/' + record[1]+'.png'
    path_load_disp_r = disp_path + '/right_pngs/' + record[1]+'.png'
    
    tmp_clr = PIL.Image.open(path_load_c_l)
    tmp_nir = PIL.Image.open(path_load_c_r).convert('L')
    
    tmp_disp_l = PIL.Image.open(path_load_disp_l)
    tmp_disp_r = PIL.Image.open(path_load_disp_r)
    
    tmp_clr = np.asarray(tmp_clr)
    tmp_nir = np.asarray(tmp_nir)
    
    tmp_disp_l = np.asarray(tmp_disp_l)
    tmp_disp_r = np.asarray(tmp_disp_r)    

    tmp_nir = tmp_nir[:, :, np.newaxis]
    tmp_nir = np.concatenate((tmp_nir, tmp_nir, tmp_nir), 2)
    
    list_x = []
    list_y = []
    
    default_x = random.randint(100, 500)
    default_y = random.randint(100, 300)

    for i in range(3):
        while default_x in list_x:
            default_x = random.randint(100, 500)
        list_x.append(default_x)
        while default_y in list_y:
            default_y = random.randint(100, 300)
        list_y.append(default_y)
    
    left_disp1 = tmp_disp_l[list_y[0], list_x[0], 0]
    right_disp1 = tmp_disp_r[list_y[0], list_x[0] - left_disp1, 0]
    left_disp2 = tmp_disp_l[list_y[1], list_x[1], 0]
    right_disp2 = tmp_disp_r[list_y[1], list_x[1] - left_disp2, 0]
    left_disp3 = tmp_disp_l[list_y[2], list_x[2], 0]
    right_disp3 = tmp_disp_r[list_y[2], list_x[2] - left_disp3, 0]
    
    if left_disp1 == right_disp1:
        left_patch = tmp_clr[list_y[0]-size:list_y[0]+size, list_x[0]-size:list_x[0]+size, :]
        right_patch = tmp_nir[list_y[0]-size:list_y[0]+size, list_x[0]-left_disp1-size:list_x[0]-left_disp1+size, :]
        imgs = [left_patch, right_patch] 
        imgs_comb = np.hstack(imgs[i] for i in range(2))
        imgs_comb = Image.fromarray(imgs_comb)
        path_save  = save_path + str(cnt) + '.jpg'
        imgs_comb.save(path_save)
        cnt = cnt + 1
    if left_disp2 == right_disp2:
        left_patch = tmp_clr[list_y[1]-size:list_y[1]+size, list_x[1]-size:list_x[1]+size, :]
        right_patch = tmp_nir[list_y[1]-size:list_y[1]+size, list_x[1]-left_disp2-size:list_x[1]-left_disp2+size, :]
        imgs = [left_patch, right_patch] 
        imgs_comb = np.hstack(imgs[i] for i in range(2))
        imgs_comb = Image.fromarray(imgs_comb)
        path_save  = save_path + str(cnt) + '.jpg'
        imgs_comb.save(path_save)
        cnt = cnt + 1
    if left_disp3 == right_disp3:
        left_patch = tmp_clr[list_y[2]-size:list_y[2]+size, list_x[2]-size:list_x[2]+size, :]
        right_patch = tmp_nir[list_y[2]-size:list_y[2]+size, list_x[2]-left_disp3-size:list_x[2]-left_disp3+size, :]
        imgs = [left_patch, right_patch] 
        imgs_comb = np.hstack(imgs[i] for i in range(2))
        imgs_comb = Image.fromarray(imgs_comb)
        path_save  = save_path + str(cnt) + '.jpg'
        imgs_comb.save(path_save)
        cnt = cnt + 1

In [ ]:
## Cell 2
## Extract negative pairs from positive pairs for training
import random
import os
import numpy as np
import PIL
from PIL import Image

path = ''
path_save = ''

file_cnt = 0
for pack in os.walk(path):
    for f in pack[2]:
        file_cnt += 1
        
print('2. Extract negative pairs from positive paris')
print('Number of positive patches: ', file_cnt)

neg = []
cnt = 0
for i in range(0, file_cnt):
    clr_patch_path  = '{}/{}.jpg'.format(path, i)
    
    default = random.randint(0, file_cnt - 1)
    while default == i:
        default = random.randint(0, file_cnt - 1)
    
    nir_patch_path  = '{}/{}.jpg'.format(path, default)
    clr_patch = PIL.Image.open(clr_patch_path)
    nir_patch = PIL.Image.open(nir_patch_path)

    neg_nir_crop = clr_patch.crop((64, 0, 128, 64))
    neg_clr_crop = nir_patch.crop((0, 0, 64, 64))
    
    neg = [neg_clr_crop, neg_nir_crop]
    
    imgs_comb = np.hstack((np.asarray(j) for j in neg))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save, cnt)
    imgs_comb.save(save_path)
    cnt += 1
    
print('Number of negative patches: ', cnt)

In [ ]:
## Cell 3
## Merge postive and negative pairs for training
import os
import numpy as np
import PIL
from PIL import Image

path_pos = ''
path_neg = ''
path_save = ''

print('3. Merge postive and negative pairs')

file_cnt_pos = 0
for pack in os.walk(path_pos):
    for f in pack[2]:
        file_cnt_pos += 1
print('   Number of positive patches: ', file_cnt_pos)

file_cnt_neg = 0
for pack in os.walk(path_neg):
    for f in pack[2]:
        file_cnt_neg += 1
print('   Number of negative patches: ', file_cnt_neg)

merge = []
cnt = 0
for i in range(0, file_cnt_pos):
    pos_patch_path  = '{}/{}.jpg'.format(path_pos, i)
    neg_patch_path  = '{}/{}.jpg'.format(path_neg, i)
    pos_patch = PIL.Image.open(pos_patch_path)
    neg_patch = PIL.Image.open(neg_patch_path)
   
    merge = [pos_patch, neg_patch]
    
    imgs_comb = np.hstack((np.asarray(j) for j in merge))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save, cnt)
    imgs_comb.save(save_path)
    cnt += 1
    
print('Number of merged patches: ', cnt)

In [ ]:
## Cell 4
## Extract positive pairs for testing
import numpy as np
import PIL
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

save_path = ''
data_path = ''
list_path = ''
folders = ['20170222_0951', '20170222_1423', '20170223_1639', '20170224_0742']
records = []
for folder in folders:
    f = open(Path(list_path) / (folder + '.txt'), 'r')
    lines = f.readlines()
    f.close()
    for i, line in enumerate(lines):
        splited = line.split()
        collection = splited[0]
        key = splited[1]
        rgb_exp = float(splited[2])
        nir_exp = float(splited[3])
        record = [collection, key, rgb_exp, nir_exp]
        records.append(record)
cnt = 0
size = 32
for record in records:
    path_load_c_l = data_path + '/' + record[0] + '/RGBResize/' + record[1] + '_RGBResize.png'
    path_load_c_r = data_path + '/' + record[0] + '/NIRResize/' + record[1] + '_NIRResize.png'
    path_load_gt = data_path + '/' + record[0] + '/Keypoint/' + record[1]+'_Keypoint.txt'
    
    tmp_clr = PIL.Image.open(path_load_c_l)
    tmp_nir = PIL.Image.open(path_load_c_r).convert('L')
    
    tmp_clr = np.asarray(tmp_clr)
    tmp_nir = np.asarray(tmp_nir)
    tmp_nir = tmp_nir[:, :, np.newaxis]
    tmp_nir = np.concatenate((tmp_nir, tmp_nir, tmp_nir), 2)

    f = open(path_load_gt, 'r')
    gts = f.readlines()
    f.close()    
    for gt in gts:
        x, y, d, c = gt.split()
        x = round(float(x) * 582) - 1
        x = int(max(0, min(582, x)))
        y = round(float(y) * 429) - 1
        y = int(max(0, min(429, y)))
        d = round(float(d) * 582)
        d = int(max(0, d))
        c = int(c)
        
        if (x-d+32)>=582 or (x-d-32)<0 or (x-size)<0 or (x+size)>=582 or (y+32)>=429 or (y-32)<0:
            continue
        else:
            rgb_patch = tmp_clr[y-size:y+size, x-size:x+size, :]
            nir_patch = tmp_nir[y-size:y+size, x-d-size:x-d+size, :]
            imgs = [rgb_patch, nir_patch] 
            imgs_comb = np.hstack(imgs[i] for i in range(2))
            imgs_comb = Image.fromarray(imgs_comb)
            path_save  = save_path + str(cnt) + '.jpg'
            imgs_comb.save(path_save)
            cnt = cnt + 1

In [ ]:
## Cell 5
## Extract negative pairs by shifting positive pairs for testing
import numpy as np
import PIL
from PIL import Image
from pathlib import Path
import random
import matplotlib.pyplot as plt

save_path = ''
data_path = ''
list_path = ''
folders = ['20170222_0951', '20170222_1423', '20170223_1639', '20170224_0742']
records = []
for folder in folders:
    f = open(Path(list_path) / (folder + '.txt'), 'r')
    lines = f.readlines()
    f.close()
    for i, line in enumerate(lines):
        splited = line.split()
        collection = splited[0]
        key = splited[1]
        rgb_exp = float(splited[2])
        nir_exp = float(splited[3])
        record = [collection, key, rgb_exp, nir_exp]
        records.append(record)
cnt = 0
size = 32
for record in records:
    path_load_c_l = data_path + '/' + record[0] + '/RGBResize/' + record[1] + '_RGBResize.png'
    path_load_c_r = data_path + '/' + record[0] + '/NIRResize/' + record[1] + '_NIRResize.png'
    path_load_gt = data_path + '/' + record[0] + '/Keypoint/' + record[1]+'_Keypoint.txt'
    
    tmp_clr = PIL.Image.open(path_load_c_l)
    tmp_nir = PIL.Image.open(path_load_c_r).convert('L')
    
    tmp_clr = np.asarray(tmp_clr)
    tmp_nir = np.asarray(tmp_nir)
    tmp_nir = tmp_nir[:, :, np.newaxis]
    tmp_nir = np.concatenate((tmp_nir, tmp_nir, tmp_nir), 2)

    f = open(path_load_gt, 'r')
    gts = f.readlines()
    f.close()    
    for gt in gts:
        x, y, d, c = gt.split()
        x = round(float(x) * 582) - 1
        x = int(max(0, min(582, x)))
        y = round(float(y) * 429) - 1
        y = int(max(0, min(429, y)))
        d = round(float(d) * 582)
        d = int(max(0, d))
        c = int(c)
        
        var = random.randint(-16, 16)
        while (var in range(-10, 11)):
            var = random.randint(-16, 16)
        
        if (x-d+32+var)>=582 or (x-d-32+var)<0 or (x-size)<0 or (x+size)>=582 or (y+32)>=429 or (y-32)<0:
            continue
        else:
            rgb_patch = tmp_clr[y-size:y+size, x-size:x+size, :]
            nir_patch = tmp_nir[y-size:y+size, x-d-size+var:x-d+size+var, :]
            imgs = [rgb_patch, nir_patch] 
            imgs_comb = np.hstack(imgs[i] for i in range(2))
            imgs_comb = Image.fromarray(imgs_comb)
            path_save  = save_path + str(cnt) + '.jpg'
            imgs_comb.save(path_save)
            cnt = cnt + 1

In [ ]:
## Cell 6
## Extract negative pairs from positive pairs for testing
import random
import os
import numpy as np
import PIL
from PIL import Image

path = ''
path_save = ''

file_cnt = 0
for pack in os.walk(path):
    for f in pack[2]:
        file_cnt += 1
        
print('2. Extract negative pairs from positive paris')
print('Number of positive patches: ', file_cnt)

neg = []
cnt = 0
for i in range(0, file_cnt):
    clr_patch_path  = '{}/{}.jpg'.format(path, i)
    
    default = random.randint(0, file_cnt - 1)
    while default == i:
        default = random.randint(0, file_cnt - 1)
    
    nir_patch_path  = '{}/{}.jpg'.format(path, default)
    clr_patch = PIL.Image.open(clr_patch_path)
    nir_patch = PIL.Image.open(nir_patch_path)

    neg_nir_crop = clr_patch.crop((64, 0, 128, 64))
    neg_clr_crop = nir_patch.crop((0, 0, 64, 64))
    
    neg = [neg_clr_crop, neg_nir_crop]
    
    imgs_comb = np.hstack((np.asarray(j) for j in neg))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save, cnt)
    imgs_comb.save(save_path)
    cnt += 1
    
print('Number of negative patches: ', cnt)

In [ ]:
## Cell 7
## Merge postive and negative pairs as follows
## 1, Merge half of postivie pairs from Cell4 and half of negative pairs from Cell5
## 2. Merge remained positive pairs and half of negative pairs from Cell6
## 3. Combine patch pairs merged in step 1 and step 2 into the test dataset folder.
import os
import numpy as np
import PIL
from PIL import Image

path_pos = ''
path_neg = ''
path_save = ''

print('3. Merge postive and negative pairs')

file_cnt_pos = 0
for pack in os.walk(path_pos):
    for f in pack[2]:
        file_cnt_pos += 1
print('   Number of positive patches: ', file_cnt_pos)

file_cnt_neg = 0
for pack in os.walk(path_neg):
    for f in pack[2]:
        file_cnt_neg += 1
print('   Number of negative patches: ', file_cnt_neg)

merge = []
cnt = 0
for i in range(0, file_cnt_pos):
    pos_patch_path  = '{}/{}.jpg'.format(path_pos, i)
    neg_patch_path  = '{}/{}.jpg'.format(path_neg, i)
    pos_patch = PIL.Image.open(pos_patch_path)
    neg_patch = PIL.Image.open(neg_patch_path)
   
    merge = [pos_patch, neg_patch]
    
    imgs_comb = np.hstack((np.asarray(j) for j in merge))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save, cnt)
    imgs_comb.save(save_path)
    cnt += 1
    
print('Number of merged patches: ', cnt)